I've found an interesting contribution to fringe fraction variability

The "orignal" code (commit https://github.com/MSLNZ/ls-fringeapp/commit/d9f5aa35f5e03e1d8b9a2f1d18dd3e40f0dfe784 on master branch)  

converts the colour image to a numpy aarray as follows
original code in master branch `FringeManager.open_image` 

```python
        img = Image.open(self.img_filename)
        img.convert("L")
        self.img_array = np.asarray(img)
        if self.img_array.ndim > 2:
            self.img_array = self.img_array.mean(axis=2)
```

This code appears to use  the PIL library 
`img.convert("L")` to convert to grey scale but the image is not converted inplace. This line should be
`img = img.convert("L")`
The code then goes on to take the mean of the RGB channels in `self.img_array`

The PIL library uses by default a weighted matrix to convert to grey scale
>  L = R * 299/1000 + G * 587/1000 + B * 114/1000

https://pc-pillow.readthedocs.io/en/latest/Image_class/Image_convert.html#image-convert

using the default
```python
img = Image.open(self.img_filename)
img = img.convert("L")
self.img_array = np.asarray(img)
```

gives differnt fringe fractions than using

```python
img = Image.open(self.img_filename)
self.img_array = np.asarray(img)
if self.img_array.ndim > 2:
    self.img_array = self.img_array.mean(axis=2)
```
see below

In [ ]:
from pathlib import Path
from PIL import Image

In [ ]:
TEST_DATA_DIR = Path(__file__).resolve().parent / "data"


def test_fringe_calc(row):
    img_filename = TEST_DATA_DIR / row[1]
    xygb = row[2:8].astype(float)
    xygb = xygb.reshape((3, 2))

    img = Image.open(img_filename)
    img = img.convert("L", matrix=(1 / 4.0, 1 / 4.0, 1 / 4.0, 1 / 4.0))
    img_array = np.asarray(img)
    # if img_array.ndim > 2:
    #     img_array = img_array.mean(axis=2)

    ffrac_calc = fp.array2frac(img_array, xygb)

    # uncertainty analysis requires 0.4e-3 at 1 sigma,
    np.testing.assert_allclose(
        ffrac_calc,
        ffrac_exp,
        atol=0.00001,
    )
